# Loading and Filtering TLE Lists

Note that, Orekit needs to be initialised first.

In [1]:
from pathlib import Path

import orekit
from orekit.pyhelpers import setup_orekit_curdir

# Init Java VM
orekit.initVM()

#  Init Orekit data
orekit_data_file_path = Path("..", "..", "data", "orekit-data", "orekit-data-reference.zip").resolve()
setup_orekit_curdir(str(orekit_data_file_path))


cdgdxfgg

the most common application is to load a file (or a string) containing TLE data in text format and filtering satellites with a certain parameter.

In [5]:
from org.orekit.time import AbsoluteDate, TimeScalesFactory

# If satkit import fails, try to locate the module
import os
try:
    import satkit
except ModuleNotFoundError:
    os.chdir(os.path.join("..", ".."))
    os.getcwd()

from satkit import process_paths
from satkit.propagation.tle_list import (
    TleRangeFilterParams,
    TleStorage,
    TleValueFilterParams,
)

# Orekit init complete, continue with loading the TLE file.

# use case 1, load from TLE file, filter for a certain satellite number
# ---------------------------------------------------------------------
alt_intermed_path = Path("docs", "tutorials")
mixed_tle_file_path_1 = Path("data", "tle_mixed_1.txt")

file_path = process_paths(alt_intermed_path, mixed_tle_file_path_1)

tle_storage_1 = TleStorage.from_path(file_path)

# Print the first element of the TLE file as an example
print(tle_storage_1.tle_list[0])

# filter for a specific satellite number
filtered_list_1 = tle_storage_1.filter_by_value(TleValueFilterParams.SAT_NR, 46495)

# Print the filtered element of the filtered TLE list
print(filtered_list_1.tle_list[0])

# use case 2, load from TLE string, filter for a certain epoch
# ---------------------------------------------------------------------
with open(file_path, "r") as f:
    tle_source_str = f.read()

tle_storage_2 = TleStorage.from_string(tle_source_str)

# filter for TLEs after a specific epoch
threshold_time = AbsoluteDate("2021-02-01T00:00:00.000", TimeScalesFactory.getUTC())
filtered_list_2 = tle_storage_2.filter_by_range(TleRangeFilterParams.EPOCH, min_value=threshold_time)

# Print the first element of the filtered TLE list
print(filtered_list_2.tle_list[0])

1 28366U 04025A   21088.48552344  .00000020  00000-0  20339-4 0  9993
2 28366  98.2305 333.6788 0106609 231.7460 127.4097 14.36733092877928
1 46495U 20068K   21089.25908752 -.00000061  00000-0  99758-7 0  9995
2 46495  97.6865  27.2569 0016962 346.9048  13.1734 15.03385776 27430
1 28366U 04025A   21088.48552344  .00000020  00000-0  20339-4 0  9993
2 28366  98.2305 333.6788 0106609 231.7460 127.4097 14.36733092877928


The example above shows a number of important functionalities. A {py:class}`.TleStorage` object can be initialised using {py:meth}`.TleStorage.from_path` method (from a TLE file) or {py:meth}`.TleStorage.from_string` method (from a string containing a list of TLEs). The latter can be useful while downloading a TLE data from a remote location without saving it to an intermediate file. The example also shows the filtering functionality, which is detailed in the next section.

Once initialised, the TLE list can be filtered using the enumerator {py:class}`.TleValueFilterParams` and a filtering value. Furthermore, to filter with an element like an identifier (e.g. name or catalogue number) where an exact match can be found, `filter_by_value` method should be used. In the example above, a satellite with the catalogue number "46495" is extracted from the list.

Conversely, if a range rather than an exact match is sought (e.g. semimajor axis, epoch or eccentricity), then `filter_by_range` method and the {py:class}`.TleRangeFilterParams` should be used. In the example above, a
threshold epoch is given and all TLE values after this threshold are extracted.

In [ ]:
# filtering with a max and min epoch date
min_threshold_time = AbsoluteDate("2021-03-29T00:00:00.000", TimeScalesFactory.getUTC())
max_threshold_time = AbsoluteDate("2021-03-29T13:00:00.000", TimeScalesFactory.getUTC())
filtered_list_3 = tle_storage_2.filter_by_range(TleRangeFilterParams.EPOCH, min_value=min_threshold_time, max_value=max_threshold_time)

# Print the first element of the filtered TLE list
print(filtered_list_3.tle_list[0])

from satkit import u

# filtering with a min inclination
min_inclination = 90 * u.deg
filtered_list_4 = tle_storage_2.filter_by_range(TleRangeFilterParams.INCLINATION, min_value=min_inclination)

# Print the first element of the filtered TLE list
print(filtered_list_4.tle_list[0])

# filtering with a max eccentricity
max_e = 0.001
filtered_list_5 = tle_storage_2.filter_by_range(TleRangeFilterParams.E, max_value=max_e)

# Print the first element of the filtered TLE list
print(filtered_list_5.tle_list[0])



In addition to the `filter_by_value` and `filter_by_range` methods, there is a third and very powerful method to filter the TLEs through user defined functions. In `filter_by_func`, a user-defined function takes the TLE, runs some test and returns `True` or `False` accordingly. For example, while TLE does not have a direct way to filter for semimajor axis, a filter can be easily written with this method.



In [ ]:
from satkit.propagation.tle import TLEUtils

# define the filter function and filter the list
def sma_filter(tle):
    """Semimajor axis filter min/max."""
    return True if 7100 * u.km > TLEUtils.compute_sma(tle) > 7000 * u.km else False


filtered_list_sma_1 = tle_storage_1.filter_by_func(sma_filter)

# Print the first element of the filtered TLE list
print(filtered_list_sma_1.tle_list[0])


The ideal way to initialise the TimeSeries is to generate a `TleStorage` from a file or another source and then filter for a unique satellite identifier.



In [ ]:
from satkit.propagation.tle_list import TleTimeSeries

tle_timeseries_1 = TleStorage.from_path(file_path).to_tle_timeseries(28366)

# Print the first element of the filtered TLE list
print(tle_timeseries_1.tle_list[0])

# Initialise through TLE data
tle_timeseries_2 = TleTimeSeries(tle_storage_1.tle_list, 28366)

Further filtering is then possible using the same methods as `TleStorage` given in the [TLE Filtering](#extracting-specific-data-from-the-lists-filtering) section. For example, all the TLEs after a certain time or above a certain semimajor axis or eccentricity value can be extracted by chaining the filters.